# Sheet 06

## Preamble

Autors: Marten Ringwelski, Nico Ostermann, Simon Liessem

Note that this notebook MUST be executed in order to get everything to work.
The tasks can't be run individually. 

Also eCampus does not allow for uploading nested directory structures which makes it hard to properly organize the files. The files are expected to be in the `data` directory which itself is placed next to this notebook.

If you extract the zip file we handed in everything should work just fine.

Autoformatting if `jupyter-black` is installed.

In [ ]:
try:
    import black
    import jupyter_black

    jupyter_black.load(
        lab=False,
        line_length=79,
        verbosity="DEBUG",
        target_version=black.TargetVersion.PY310,
    )
except ImportError:
    pass

Import all we weed and more.

Set seaborn default theme

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.feature_selection import f_classif, SelectKBest
import math as m
import plotly.express as px
import sklearn.manifold
import sklearn.discriminant_analysis
import scipy as sp
import scipy.sparse
import skimage
from dash import Dash, html, dcc, callback, Output, Input, State
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import graphviz

from matplotlib.colors import rgb2hex

Set seaborn default theme

In [ ]:
sns.set_theme()

If needed tweak parameters of matplotlib.
Here we increase the size and dpi to bet a bigger but still high-res image.

In [ ]:
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["figure.figsize"] = (20, 15)
%matplotlib inline

Disable future warnings as we get a lot of them and don't really care for this sheet.

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

# Exercise 1

### a)

First we read the Breast Cancer Dataset. Then we fill the Nans and compute the pearson correlations.

In [ ]:
df = pd.read_excel("data/breast-cancer-wisconsin.xlsx")
df = df.fillna(df.mean())
correlation = df.corr(method="pearson")

### b)

Now we create a graph with the variables as the nodes. 
The graph is strict because we dont want to have more than one edge from one node to the other.
As engine we use fdp for a force directed layout.

In [ ]:
e = graphviz.Graph("escapes", strict=True, engine="fdp")
for column in correlation.columns:
    e.node(column, column)

Now we create edges if the correlation of two nodes is greater that 0.6.

In [ ]:
for column in correlation.columns:
    for row in correlation.index:
        if correlation[column].loc[[row]].values > 0.6 and row != column:
            e.edge(column, row)

In [ ]:
e

### c)

Now we modify the edges, so that the edges are more transparent when the correlation is smaller.

In [ ]:
cmap = mpl.colormaps["Blues"]

for column in correlation.columns:
    for row in correlation.index:
        if correlation[column].loc[[row]].values > 0.6 and row != column:
            normalized_correlation = (
                correlation[column].loc[[row]].values[0] - 0.6
            ) / 0.4
            color = cmap(normalized_correlation)
            color = mpl.colors.rgb2hex(color[:3])
            e.edge(
                column,
                row,
                color=color,
                penwidth=f"{normalized_correlation*5+2}",
            )

In [ ]:
e

### d)

In this task we will use a circular layout.

In [ ]:
e = graphviz.Graph("escapes", strict=True, engine="circo")

Now we make 4 sets of nodes depending on their highest correlation and color them different.

In [ ]:
list_6 = []
list_8 = []
list_9 = []
node_cmap = mpl.colormaps["Greens"]
for column in correlation.columns:
    e.node(column, column, style="filled", fillcolor="white")
    for row in correlation.index:
        if correlation[column].loc[[row]].values[0] > 0.6 and column != row:
            list_6.append(column)
        if correlation[column].loc[[row]].values[0] > 0.8 and column != row:
            list_8.append(column)
        if correlation[column].loc[[row]].values[0] > 0.9 and column != row:
            list_9.append(column)

for column in correlation.columns:
    if column in list_9:
        e.node(
            column,
            column,
            style="filled",
            fillcolor=rgb2hex(node_cmap(0.9)[:3]),
        )
    elif column in list_8:
        e.node(
            column,
            column,
            style="filled",
            fillcolor=rgb2hex(node_cmap(0.6)[:3]),
        )
    elif column in list_6:
        e.node(
            column,
            column,
            style="filled",
            fillcolor=rgb2hex(node_cmap(0.3)[:3]),
        )

Here we have to insert the edges again, because the nodes were generated new.

In [ ]:
cmap = mpl.cm.get_cmap("Blues")

for column in correlation.columns:
    for row in correlation.index:
        if correlation[column].loc[[row]].values > 0.6 and row != column:
            normalized_correlation = (
                correlation[column].loc[[row]].values[0] - 0.6
            ) / 0.4
            color = cmap(normalized_correlation)
            color = mpl.colors.rgb2hex(color[:3])
            e.edge(
                column,
                row,
                color=color,
                penwidth=f"{normalized_correlation*5+2}",
            )

In [ ]:
e

### e)

The nodes that are disconected are mitoses and code. 
This makes sense because the code is metadata and it makes sense, that this metadata does not correlate.
Mitosis denotes a value of cell division.

In the picture we see that thickness-class are strongly correlated and class-epiCelSize are strongly correlated.
Nevertheless thickness-epiCelSize are not stronly correlated.

We would take bareNuc, uniCelShape and uniCelS since they are all very strongly correlated to class.



# Exercise 2

### a)

Vertices: 
The first two integers, and last two integers give the beginning and end of a double linked list with outgoing, incoming edges of the vertex, respectively.
Edges:
The first 4 integers store the next edge and the previous edge, for each of the double linked lists.
The last two integers store the first vertex (source) and second vertex (target)
    
To reduce memory consumption, the double linked lists can be replaced with single linked lists.
This leads to the second and forth integer of the vertex, and the second and forth integer of the edge to be omittable.


### b)

In the case of the graph being a directed Graph, we assume the desired edge has u as its first and v as its second vertex. 

Vertex u

for edge in u.outgoing_edges: 
***
    if edge.second_edge == v: 
***
        return edge.index
***
return None

### c)

The modifications serve as a way to further spread the choosing of the pivot elements by cutting the cost of all edges within the chosen path between two pivot elements in half.
This serves the purpose of a wider spread selection of pivot elements, instead of the pivot elements clustering within the area of the first two.

### d)

Geometric zoom: zooming closer to the actual target, i.e. making a vertex bigger

Detail zoom: increasing the level of detail on a given target

# Exercise 3

### a)

Read data and use mean for missing data.

In [ ]:
df = pd.read_excel(
    "data/Data_Cortex_Nuclear.xls",
    index_col="MouseID",
)
df = df.fillna(df.mean())

Now we do PCA, t-SNE and ISOMAP with parameters that we used in sheet 04.

In [ ]:
meta_columns = ["Genotype", "Treatment", "Behavior", "class"]
data_columns = df.columns.difference(meta_columns)
df_wo_meta = df[data_columns]

df_scs = df_wo_meta[
    np.logical_or(
        df["class"] == "c-SC-s",
        df["class"] == "t-SC-s",
    )
].copy()

In [ ]:
pca = sk.decomposition.PCA(
    n_components=2,
)
# XXX There must be a better way to do this
df_pca = pd.DataFrame(
    pca.fit_transform(df_scs),
    columns=["x", "y"],
    index=df_scs.index,
)
df_pca[df.columns] = df

In [ ]:
# We don't care that the calculation is expensive
warnings.simplefilter(
    action="ignore",
    category=sp.sparse.SparseEfficiencyWarning,
)

isomap = sk.manifold.Isomap(
    n_neighbors=20,
)

df_isomap = pd.DataFrame(
    isomap.fit_transform(df_scs),
    columns=["x", "y"],
    index=df_scs.index,
)

df_isomap[df.columns] = df

In [ ]:
tsne = sk.manifold.TSNE(
    perplexity=15,
)

df_tsne = pd.DataFrame(
    tsne.fit_transform(df_scs),
    columns=["x", "y"],
    index=df_scs.index,
)

df_tsne[df.columns] = df

Now create the dash app.

In [ ]:
# https://dash.plotly.com/workspaces/using-dash-in-jupyter-and-workspaces

In [ ]:
app = JupyterDash(__name__)

In [ ]:
app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.Div(
                    children=[
                        # First Graph
                        dcc.RadioItems(
                            ["PCA", "ISOMAP", "t-SNE"],
                            "PCA",
                            id="radio-items-dim-reduction",
                        ),
                        dcc.Graph(
                            figure={},
                            id="graph-dim-reduction",
                        ),
                    ],
                    className="column",
                ),
                html.Div(
                    children=[
                        # Second Graph
                        dcc.Dropdown(
                            data_columns,
                            data_columns[0],
                            id="dropdown-x-axis",
                        ),
                        dcc.Dropdown(
                            data_columns,
                            data_columns[0],
                            id="dropdown-y-axis",
                        ),
                        dcc.Graph(
                            figure={},
                            id="graph-xy",
                        ),
                    ],
                    className="column",
                ),
            ],
            className="row",
        ),
        # Button for adding things
        html.Button(
            "Add",
            id="button-add",
        ),
        html.Div(children=[], id="div-user-defined-graphs"),
    ],
)

First add the radio callback as subtask a) asked us to.

In [ ]:
@app.callback(
    Output(
        component_id="graph-dim-reduction",
        component_property="figure",
    ),
    Input(
        component_id="radio-items-dim-reduction",
        component_property="value",
    ),
)
def update_graph_dim_reduction(dim_reduction):
    df = None
    if dim_reduction == "PCA":
        df = df_pca
    elif dim_reduction == "ISOMAP":
        df = df_isomap
    elif dim_reduction == "t-SNE":
        df = df_tsne
    else:
        assert False, "Nothing selected"

    fig = px.scatter(
        df,
        x="x",
        y="y",
    )
    return fig

Now add a callback to update the second graph.

In [ ]:
@app.callback(
    Output(
        component_id="graph-xy",
        component_property="figure",
    ),
    Input(
        component_id="dropdown-x-axis",
        component_property="value",
    ),
    Input(
        component_id="dropdown-y-axis",
        component_property="value",
    ),
)
def update_graph_xy(x_axis, y_axis):
    assert x_axis is not None, "Nothing selected"
    assert y_axis is not None, "Nothing selected"

    fig = px.scatter(
        df,
        x=x_axis,
        y=y_axis,
    )
    return fig

Now define the callback for the button.

In [ ]:
@app.callback(
    Output(
        component_id="div-user-defined-graphs",
        component_property="children",
    ),
    Input("button-add", "n_clicks"),
    State(
        component_id="dropdown-x-axis",
        component_property="value",
    ),
    State(
        component_id="dropdown-y-axis",
        component_property="value",
    ),
    State(
        component_id="div-user-defined-graphs",
        component_property="children",
    ),
    # We don't want to show a graph when the user didn't click.
    prevent_initial_call=True,
)
def update_div_user_defined_graphs(_, x_axis, y_axis, children):
    assert x_axis is not None, "Nothing selected"
    assert y_axis is not None, "Nothing selected"

    # Probably it would be better to just copy the figure from above, but anyways
    fig = px.scatter(
        df,
        x=x_axis,
        y=y_axis,
    )
    graph = dcc.Graph(figure=fig)
    children.append(graph)
    return children

Now run the thing.

In [ ]:
if __name__ == "__main__":
    app.run_server()

### b)

It made sense to combine all tasks in one so pleas have a look at a) for all solutions for a), b) and c).

### c)

It made sense to combine all tasks in one so pleas have a look at a) for all solutions for a), b) and c).